In [ ]:
import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

import vector

import sys

vector.register_awkward()

import coffea

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import itertools
from itertools import combinations

import nanoaod_analysis_tools as nat

import time

In [ ]:
print(f"python: {sys.version}\n")

print(f"numpy:   {np.__version__}")
print(f"uproot:  {uproot.__version__}")
print(f"awkward: {ak.__version__}")
print(f"vector:  {vector.__version__}")
print()

print(f"coffea:  {coffea.__version__}")


In [ ]:
data_dir = '/home/bellis/top_data/NANOAOD/'

# Laptop
#infilename = 'small_skims_10k/TT_TToBCE_TuneCP5_BNV_2018_SMALL_10k.root'
#infilename = 'TTToHadronic_UL_2018_SMALL_100k.root'

# Beth Harmon
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'
#infilename = 'TTToSemiLeptonic_UL_2018.root' # Also desktop home computer
infilename = 'Reza_signal/nAOD_step_BNV_TT_TSUE/NAOD-00000_190.root' # Also desktop home computer, laptop

# Desktop home computer
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'


infile = uproot.open(data_dir + infilename)

print(f"Reading in {infilename}") 
dataset_type, mc_type, trigger, topology, year = nat.extract_dataset_type_and_trigger_from_filename(infilename) 
print(f"input file information:  dataset type: {dataset_type}   MC type: {mc_type}  trigger: {trigger}  topology: {topology}")

In [ ]:
print("Processing data...") 

start = time.time()

events = NanoEventsFactory.from_root(data_dir + infilename, schemaclass=NanoAODSchema).events()

print(len(events))

genpart = events.GenPart

topology = f"had_{topology}" 

verbose = True
match_first = True

match_first_tag = ""
if match_first:
    match_first_tag = "_FIRST_MATCH"

event_truth_indices, truth_indices = nat.truth_matching_identify_genpart(genpart,topology=topology,verbose=verbose, match_first=match_first) 

truth_filename = f"TRUTH_INFORMATION{match_first_tag}_{infilename.split('/')[-1].split('.root')[0]}.npz" 

np.savez(truth_filename,event_truth_indices=event_truth_indices,truth_indices=truth_indices,allow_pickle=False)


print(f"time to process: {time.time()-start} seconds")

In [ ]:
################################################################################
# Generate the indices for the diferent combinations
################################################################################
def generate_event_topology_indices(njets,nleps,verbose=False):

    index_combinations = []

    if njets<5 or nleps<1:
        return [[None,None,None,None,None,None]]

    jetindices = np.arange(njets,dtype=int)
    lepindices = np.arange(nleps,dtype=int)

    x = combinations(jetindices,3)

    for had in x:
        remaining = np.delete(jetindices, np.argwhere( (jetindices==had[0]) | (jetindices==had[1]) | (jetindices==had[2]) ))
        bnv = combinations(remaining,2)
        for b in bnv:
            for lep in lepindices:
                index_combinations.append([had[0],had[1],had[2],b[0],b[1],lep])

                if verbose:
                    print(had,b,lep)

    return np.array(index_combinations)


In [ ]:
truth_data = np.load(truth_filename, allow_pickle=False)

truth_indices = truth_data['truth_indices']
event_truth_indices = truth_data['event_truth_indices']

In [ ]:
print(event_truth_indices)
print()
print(truth_indices)
print()

print(event_truth_indices.size, truth_indices.shape)

In [ ]:
#events[event_truth_indices].fields

print(len(events))
print(len(events[event_truth_indices]))

muons = events[event_truth_indices].Muon
genparts = events[event_truth_indices].GenPart

In [ ]:
len(genparts)

g = genparts[0][truth_indices][0]
print(g)
for a in g:
    print(a.pdgId)
    
print(g.fields)

In [ ]:
genparts = events.GenPart
jets = events.Jet

gps = ak.zip({
    "pt": genparts.pt,
    "phi": genparts.phi,
    "eta": genparts.eta,
    "status": genparts.status,
    "pdgId": genparts.pdgId,
}, with_name="Momentum4D")

jts = ak.zip({
    "pt": jets.pt,
    "phi": jets.phi,
    "eta": jets.eta,
    "mass": jets.mass,
    "btag": jets.btagDeepB,
}, with_name="Momentum4D")


print(genparts)
print(gps)
print(jts)

In [ ]:
gps

In [ ]:
#gps.deltaR(jts)

In [ ]:
print(event_truth_indices.size, truth_indices.shape)

genparts = events.GenPart

gen_hadb = genparts[(event_truth_indices, truth_indices.transpose()[0])]
gen_hadWq1 = genparts[(event_truth_indices, truth_indices.transpose()[1])]
gen_hadWq2 = genparts[(event_truth_indices, truth_indices.transpose()[2])]
gen_bnvq1 = genparts[(event_truth_indices, truth_indices.transpose()[3])]
gen_bnvq2 = genparts[(event_truth_indices, truth_indices.transpose()[4])]
gen_bnvlep = genparts[(event_truth_indices, truth_indices.transpose()[5])]


In [ ]:
#gen_hadb.nearest(events[event_truth_indices].Jet)
#events[event_truth_indices].Jet

In [ ]:
matched_jet = genparts[event_truth_indices].nearest(events[event_truth_indices].Jet)

matched_lep = genparts[event_truth_indices].nearest(events[event_truth_indices].Electron)

In [ ]:
# Need this for the index of each of our matched_jets
# Because we masked, we go from 0, 1, 2, ....
event_idx = np.arange(0,len(truth_indices))

matched_hadb = matched_jet[(event_idx, truth_indices.transpose()[0])]
matched_hadWq1 = matched_jet[(event_idx, truth_indices.transpose()[1])]
matched_hadWq2 = matched_jet[(event_idx, truth_indices.transpose()[2])]
matched_bnvq1 = matched_jet[(event_idx, truth_indices.transpose()[3])]
matched_bnvq2 = matched_jet[(event_idx, truth_indices.transpose()[4])]
matched_bnvlep = matched_lep[(event_idx, truth_indices.transpose()[5])]

In [ ]:
for i in range(0,10):
    print(gen_hadb[i].pt, matched_hadb[i].pt, "   ", gen_hadb[i].eta, matched_hadb[i].eta, "    ", gen_hadb[i].phi, matched_hadb[i].phi)

In [ ]:
vec_gen_hadb = vector.Array(gen_hadb)
vec_gen_hadWq1 = vector.Array(gen_hadWq1)
vec_gen_hadWq2 = vector.Array(gen_hadWq2)

vec_gen_bnvq1 = vector.Array(gen_bnvq1)
vec_gen_bnvq2 = vector.Array(gen_bnvq2)
vec_gen_bnvlep = vector.Array(gen_bnvlep)


#vec_gen_hadb

vec_jets = vector.Array(events[event_truth_indices].Jet)
vec_leps = vector.Array(events[event_truth_indices].Electron)


#dR = vec_gen_hadb.deltaR(vec_jets)
#dRlist = dR.tolist()

#dpt = np.abs(vec_gen_hadb.pt - vec_jets.pt)

#print(dpt[0])

In [ ]:
def match_jets_with_partons(vec_parton, vec_jets):
    
    dR = vec_parton.deltaR(vec_jets)
    dRlist = dR.tolist()
    
    minvals = ak.min(dR,axis=1)
    
    matched_indices = []
    N = len(vec_parton)

    for i in range(N):
        if i%10000==0:
            print(i)
        #print(dRlist[i], minvals[i])
        idx = None
        
        # Sometimes there might not be a minvals because there were no
        # reconstructed jets or muons or electrons
        if minvals[i] is None:
            idx = -1 # NEED TO DO SOMETHING BETTER
        elif minvals[i]<10000.4:
            idx = dRlist[i].index(minvals[i])
            #idx = dR[i].tolist().index(a)
        matched_indices.append(idx)
    
    return dR,np.array(matched_indices)

In [ ]:
ak.min(vec_gen_bnvlep.deltaR(vec_leps),axis=1)

In [ ]:
#'''
# midx is matched-index
dR1,midx_hadb = match_jets_with_partons(vec_gen_hadb, vec_jets)
dR2,midx_hadWq1 = match_jets_with_partons(vec_gen_hadWq1, vec_jets)
dR3,midx_hadWq2 = match_jets_with_partons(vec_gen_hadWq2, vec_jets)
dR4,midx_bnvq1 = match_jets_with_partons(vec_gen_bnvq1, vec_jets)
dR5,midx_bnvq2 = match_jets_with_partons(vec_gen_bnvq2, vec_jets)

dR6,midx_bnvlep = match_jets_with_partons(vec_gen_bnvlep, vec_leps)

#mask = (midx_hadb>=0) & (midx_hadWq1>=0) & (midx_hadWq2>=0) & \
#       (midx_bnvlep>=0) & (midx_bnvq1>=0) & (midx_bnvq2>=0)

#event_idx = event_idx[mask]

matched_hadb = vec_jets[(event_idx,midx_hadb)]#[mask])]
matched_hadWq1 = vec_jets[(event_idx,midx_hadWq1)]#[mask])]
matched_hadWq2 = vec_jets[(event_idx,midx_hadWq2)]#[mask])]

matched_bnvq1 = vec_jets[(event_idx,midx_bnvq1)]#[mask])]
matched_bnvq2 = vec_jets[(event_idx,midx_bnvq2)]#[mask])]
matched_bnvlep = vec_leps[(event_idx,midx_bnvlep)]#[mask])]


'''
vec_gen_hadb = vec_gen_hadb[mask]
vec_gen_hadWq1 = vec_gen_hadWq1[mask]
vec_gen_hadWq2 = vec_gen_hadWq2[mask]

vec_gen_bnvq1 = vec_gen_bnvq1[mask]
vec_gen_bnvq2 = vec_gen_bnvq2[mask]
vec_gen_bnvlep = vec_gen_bnvlep[mask]
'''

In [ ]:
x = ak.Array([0, 1, 2, 3, 4])
mask = x>2
print(mask)

print(x[mask])
print(x.mask[mask])

In [ ]:
#midx
#vec_leps[([0,1],[0,None])]
bad_nones = midx >= 0

In [ ]:
# Check

'''
icount = 0
for g,j in zip(vec_gen_hadb, matched_hadb):
    print(f"{g.pt:7.3f} {j.pt:7.3f} {np.abs(g.pt - j.pt)/g.pt:7.3f}   {g.eta:7.3f} {j.eta:7.3f}     {g.phi:7.3f} {j.phi:7.3f}   {g.deltaR(j):7.4f}")
    
    if icount>20:
        break
    
    icount += 1
'''

In [ ]:
plt.hist(ak.min(dR1,axis=1),bins=100,range=(0,2.9));
plt.hist(ak.min(dR2,axis=1),bins=100,range=(0,2.9));
plt.hist(ak.min(dR3,axis=1),bins=100,range=(0,2.9));

In [ ]:
plt.figure()
dpt = vec_gen_hadb.pt - matched_hadb.pt
plt.hist(dpt,bins=100,range=(-100,100));

plt.figure()
dpt = vec_gen_hadWq1.pt - matched_hadWq1.pt
plt.hist(dpt,bins=100,range=(-100,100));

plt.figure()
dpt = vec_gen_hadWq2.pt - matched_hadWq2.pt
plt.hist(dpt,bins=100,range=(-100,100));

In [ ]:
#minvals = ak.min(dR,axis=1)#.tolist()
#
#print(dR[0])

In [ ]:
gen_top = vec_gen_hadb + vec_gen_hadWq1 + vec_gen_hadWq2

plt.hist(gen_top.mass,bins=100,range=(50,300));

In [ ]:
x = vec_gen_bnvlep.deltaR(matched_bnvlep)

plt.hist(x,bins=100, range=(0,0.02));

In [ ]:
mask = (vec_gen_bnvlep.deltaR(matched_bnvlep)<0.4) & \
       (vec_gen_bnvq1.deltaR(matched_bnvq1)<0.4) & \
       (vec_gen_bnvq2.deltaR(matched_bnvq2)<0.4) & \
       ((vec_gen_bnvlep.pt - matched_bnvlep.pt)/vec_gen_bnvlep.pt<3) & \
       ((vec_gen_bnvq1.pt - matched_bnvq1.pt)/vec_gen_bnvq1.pt<3) & \
       ((vec_gen_bnvq2.pt - matched_bnvq2.pt)/vec_gen_bnvq2.pt<3) & \
       (vec_gen_hadb.deltaR(matched_hadb)<0.4) & \
       (vec_gen_hadWq1.deltaR(matched_hadWq1)<0.4) & \
       (vec_gen_hadWq2.deltaR(matched_hadWq2)<0.4) & \
       ((vec_gen_hadb.pt - matched_hadb.pt)/vec_gen_hadb.pt<3) & \
       ((vec_gen_hadWq1.pt - matched_hadWq1.pt)/vec_gen_hadWq1.pt<3) & \
       ((vec_gen_hadWq2.pt - matched_hadWq2.pt)/vec_gen_hadWq2.pt<3)
    
    
    
gen_top = matched_hadb + matched_hadWq1 + matched_hadWq2
#gen_top = matched_hadWq1 + matched_hadWq2


print(len(gen_top.mass))
print(len(gen_top.mass[mask]))

plt.hist(gen_top.mass[mask],bins=100,range=(0,300));
plt.plot([173,173],[0,1000],'k--')

In [ ]:
'''
mask = (vec_gen_bnvlep.deltaR(matched_bnvlep)<0.4) & \
       (vec_gen_bnvq1.deltaR(matched_bnvq1)<0.4) & \
       (vec_gen_bnvq2.deltaR(matched_bnvq2)<0.4) & \
       ((vec_gen_bnvlep.pt - matched_bnvlep.pt)/vec_gen_bnvlep.pt<3) & \
       ((vec_gen_bnvq1.pt - matched_bnvq1.pt)/vec_gen_bnvq1.pt<3) & \
       ((vec_gen_bnvq2.pt - matched_bnvq2.pt)/vec_gen_bnvq2.pt<3)
'''
gen_top = matched_bnvlep + matched_bnvq1 + matched_bnvq2
#gen_top = matched_hadWq1 + matched_hadWq2

print(len(gen_top.mass))
print(len(gen_top.mass[mask]))


plt.hist(gen_top.mass[mask],bins=100,range=(0,300));
plt.plot([173,173],[0,1000],'k--')

In [ ]:
x = matched_jet[(np.arange(0,len(truth_indices)),truth_indices.transpose()[0])]

print(x[0].pt, x[0].eta, x[0].phi)

g = genparts[(event_truth_indices, truth_indices.transpose()[0])]
print(g[0].pt, g[0].eta, g[0].phi)


In [ ]:
b = matched_hadb[truth_indices.transpose()[0]]

print(matched_hadb, len(matched_hadb))

print(b,len(b))

print(b[0])

In [ ]:
len(matched_jet[0][truth_indices[0]])

In [ ]:
for m in matched_jet[0][truth_indices[0]]:
    print(m.pt)

In [ ]:

#genparts = events.GenPart
#jets = events.Jet

#nearjets = genparts.nearest(jets)

'''
icount = 0
for i in g:
    print(i)
    for j in range(len(i)):
        print(i[j].pt)
    icount += 1

    if icount>10:
        break
'''        
'''
for i in range(0,10):
    print("----")
    eidx = event_truth_indices[i]
    gidx = truth_indices[i]
    for j,jet in enumerate(nearjets[eidx][gidx]):
        gen = genparts[eidx][gidx][j]
        dR = gen.deltaR(jet)
        print(gen.pt,jet.pt)

    #genparts[eidx].nearest(events.Jet[eidx])
'''

In [ ]:
#-------------------------------#

njet = infile["Events"]["nJet"].array()#[0:1000]
nmuon = infile["Events"]["nMuon"].array()#[0:1000]

min_njets = 5
max_njets = 8

mask_njets = (njet>=min_njets) & (njet<=max_njets)
#-------------------------------#


muon_branch_arrays = infile["Events"].arrays(filter_name="Muon_*")#[0:1000]
jet_branch_arrays = infile["Events"].arrays(filter_name="Jet_*")#[0:1000]

muons = ak.zip({
    "pt": muon_branch_arrays[mask_njets]["Muon_pt"],
    "phi": muon_branch_arrays[mask_njets]["Muon_phi"],
    "eta": muon_branch_arrays[mask_njets]["Muon_eta"],
    "mass": muon_branch_arrays[mask_njets]["Muon_mass"],
    "charge": muon_branch_arrays[mask_njets]["Muon_charge"],
}, with_name="Momentum4D")

jets = ak.zip({
    "pt": jet_branch_arrays[mask_njets]["Jet_pt"],
    "phi": jet_branch_arrays[mask_njets]["Jet_phi"],
    "eta": jet_branch_arrays[mask_njets]["Jet_eta"],
    "mass": jet_branch_arrays[mask_njets]["Jet_mass"],
    "btagDeepB": jet_branch_arrays[mask_njets]["Jet_btagDeepB"],
}, with_name="Momentum4D")

jet_combos = ak.combinations(jets, 5)
#muon_combos = ak.combinations(muons, 1)
muon_combos = ak.zip((muons,))

jet1, jet2, jet3, jet4, jet5 = ak.unzip(jet_combos)
muon1 = ak.unzip(muon_combos)

### Need this for later

njet_masked = infile["Events"]["nJet"].array()[mask_njets]#[0:1000]
nmuon_masked = infile["Events"]["nMuon"].array()[mask_njets]#[0:1000]

In [ ]:
print(len(njet), len(njet_masked))

In [ ]:
# Generate the combinations

combos = []
event_indices = []

icount = 0
for i,(nj,nm) in enumerate(zip(njet_masked,nmuon_masked)):
    
    combo = generate_event_topology_indices(njets=nj,nleps=nm)

    if combo[0][0] is None:
        continue
    
    for c in combo:
        combos.append(np.array(c))
    event_indices += (i*np.ones(len(combo),dtype=int)).tolist()
    icount += 1

combos = np.array(combos)

print(combos.shape)

In [ ]:
print(event_indices[0:10])
print(combos[0:10])

print()
print(len(event_indices), combos.shape)

In [ ]:
jet1 = jets[(event_indices,combos.transpose()[0])]
jet2 = jets[(event_indices,combos.transpose()[1])]
jet3 = jets[(event_indices,combos.transpose()[2])]
jet4 = jets[(event_indices,combos.transpose()[3])]
jet5 = jets[(event_indices,combos.transpose()[4])]

muon = muons[(event_indices,combos.transpose()[5])]

In [ ]:
def top_variables(jets, decay_type='had'):    
    
    # btag
    var1 = jets[0].btagDeepB
    var2 = jets[1].btagDeepB
    var3 = None
    if decay_type == 'had':
        var3 = jets[2].btagDeepB
    elif decay_type == 'bnv':
        var3 = jets[2].charge
        
    # Boost to CM of 3-jet (or 2-jet + lepton) system
    top_p4 = jets[0] + jets[1] + jets[2]
    boost_p4 = top_p4
    
    boost_p4 = ak.with_field(boost_p4, 173.0, "tau")

    jet1_boosted = jets[0].boostCM_of(boost_p4)
    jet2_boosted = jets[1].boostCM_of(boost_p4)
    jet3_boosted = jets[2].boostCM_of(boost_p4)

    # Need to do this so we only sort the 2 jets for bnv decays
    if decay_type=='had':

        # Sort things by the magnitude of momentum in the top-CM frame    
        sort_by = np.array([jet1_boosted.mag, jet2_boosted.mag, jet3_boosted.mag]).transpose()
        idx = np.argsort(sort_by)

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = sort_by
        jet3_boosted_mag,jet2_boosted_mag,jet1_boosted_mag = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jets[0], jets[1], jets[2]]).transpose()
        jet3,jet2,jet1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()


        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jet1_boosted, jet2_boosted, jet3_boosted]).transpose()
        jet3_boosted,jet2_boosted,jet1_boosted = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([var1, var2, var3]).transpose()
        var3, var2, var1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Convert back to awkward
        jet1 = ak.Array(jet1,with_name="Momentum4D")
        jet2 = ak.Array(jet2,with_name="Momentum4D")
        jet3 = ak.Array(jet3,with_name="Momentum4D")

        jet1_boosted = ak.Array(jet1_boosted,with_name="Momentum4D")
        jet2_boosted = ak.Array(jet2_boosted,with_name="Momentum4D")
        jet3_boosted = ak.Array(jet3_boosted,with_name="Momentum4D")

        '''
        #jet1_pt,jet2_pt,jet3_pt = jet1.pt,jet2.pt,jet3.pt
        print(type(jet1))
        print(jet1.fields)
        jet1_pt = jet1.rho # This is pt
        jet2_pt = None
        jet3_pt = None
        #array_to_sort = np.array([jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt]).transpose()
        jet1_boosted_pt,jet2_boosted_pt,jet3_boosted_pt = jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt#np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        had_dR12 = jet1.deltaR(jet2)
        had_dR13 = jet1.deltaR(jet3)
        had_dR23 = jet2.deltaR(jet3)
        had_dR1_23 = jet1.deltaR(jet2 + jet3)

        had_p12 = jet1 + jet2
        had_p13 = jet1 + jet3
        had_p23 = jet2 + jet3

        had_dThetaCM12 = jet1_boosted.deltaangle(jet2_boosted)
        had_dThetaCM13 = jet1_boosted.deltaangle(jet3_boosted)
        had_dThetaCM23 = jet2_boosted.deltaangle(jet3_boosted)
        had_dThetaCM1_23 = jet1_boosted.deltaangle(jet2_boosted + jet3_boosted)
        '''
        
    elif decay_type=='bnv':
   
        # Sort things by the magnitude of momentum in the top-CM frame    
        sort_by = np.array([jet1_boosted.mag, jet2_boosted.mag]).transpose()
        idx = np.argsort(sort_by)

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = sort_by
        jet2_boosted_mag,jet1_boosted_mag = np.take_along_axis(array_to_sort, idx, axis=1).transpose()
        jet3_boosted_mag = jet3_boosted.mag

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jets[0], jets[1]]).transpose()
        jet2,jet1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()
        jet3 = jets[2]

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jet1_boosted, jet2_boosted]).transpose()
        jet2_boosted,jet1_boosted = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([var1, var2]).transpose()
        var2, var1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Convert back to awkward
        jet1 = ak.Array(jet1,with_name="Momentum4D")
        jet2 = ak.Array(jet2,with_name="Momentum4D")
        jet3 = ak.Array(jet3,with_name="Momentum4D")

        jet1_boosted = ak.Array(jet1_boosted,with_name="Momentum4D")
        jet2_boosted = ak.Array(jet2_boosted,with_name="Momentum4D")
        jet3_boosted = ak.Array(jet3_boosted,with_name="Momentum4D")

    #jet1_pt,jet2_pt,jet3_pt = jet1.pt,jet2.pt,jet3.pt
    #print(type(jet1))
    #print(jet1.fields)
    jet1_pt = jet1.rho # This is pt
    jet2_pt = jet1.rho
    jet3_pt = jet1.rho
    #array_to_sort = np.array([jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt]).transpose()
    jet1_boosted_pt,jet2_boosted_pt,jet3_boosted_pt = jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt#np.take_along_axis(array_to_sort, idx, axis=1).transpose()

    dR12 = jet1.deltaR(jet2)
    dR13 = jet1.deltaR(jet3)
    dR23 = jet2.deltaR(jet3)
    dR1_23 = jet1.deltaR(jet2 + jet3)
    dR3_12 = jet3.deltaR(jet1 + jet2)


    p12 = jet1 + jet2
    p13 = jet1 + jet3
    p23 = jet2 + jet3

    dThetaCM12 = jet1_boosted.deltaangle(jet2_boosted)
    dThetaCM13 = jet1_boosted.deltaangle(jet3_boosted)
    dThetaCM23 = jet2_boosted.deltaangle(jet3_boosted)
    dThetaCM1_23 = jet1_boosted.deltaangle(jet2_boosted + jet3_boosted)
    dThetaCM3_12 = jet3_boosted.deltaangle(jet1_boosted + jet2_boosted)

    #xsorted,ysorted,zsorted = xsort.transpose()
    
    return top_p4.mass, p12.mass, p13.mass, p23.mass, \
           dR12, dR13, dR23, dR1_23, dR3_12,\
           dThetaCM12, dThetaCM13, dThetaCM23, dThetaCM1_23, dThetaCM3_12, \
           jet1_boosted_pt, jet2_boosted_pt, jet3_boosted_pt, \
           jet1_boosted_mag, jet2_boosted_mag, jet3_boosted_mag, \
           var1, var2, var3


def event_hypothesis(jets, lepton):
    had_p4 = jets[0] + jets[1] + jets[2]
    bnv_p4 = jets[3] + jets[4] + lepton
    
    angle = had_p4.deltaangle(bnv_p4)

    had_variables = top_variables([jets[0], jets[1], jets[2]], decay_type='had')
    print("Calculated the hadronic variables")
    #print(type(jets[3]))
    bnv_variables = top_variables([jets[3], jets[4], lepton], decay_type='bnv')
    print("Calculated the BNV variables")
    
    return had_variables, bnv_variables, angle



In [ ]:
#x = vector.Array(matched_hadb)
x = vector.Array({"eta":matched_hadb.eta,\
                  "rho":matched_hadb.rho, \
                  "phi": matched_hadb.phi, \
                  "mass":matched_hadb.tau, \
                  "btagDeepB":matched_hadb.btagDeepB} \
                 , with_name="Momentum4D")

y = vector.Array({"eta":matched_hadWq1.eta,\
                  "rho":matched_hadWq1.rho, \
                  "phi": matched_hadWq1.phi, \
                  "mass":matched_hadWq1.tau, \
                  "btagDeepB":matched_hadWq1.btagDeepB} \
                 , with_name="Momentum4D")

z = vector.Array({"eta":matched_hadWq2.eta,\
                  "rho":matched_hadWq2.rho, \
                  "phi": matched_hadWq2.phi, \
                  "mass":matched_hadWq2.tau, \
                  "btagDeepB":matched_hadWq2.btagDeepB} \
                 , with_name="Momentum4D")

had_variables = top_variables([x,y,z], decay_type='had')

x

In [ ]:
had_variables

In [ ]:
#matched_hadb.fields
#matched_bnvlep.fields
print(type(matched_hadb))
print(type(jet1))

len(jet1)
len(matched_hadb)

In [ ]:
#had_variables, bnv_variables, angle = event_hypothesis([jet1, jet2, jet3, jet4, jet5], muon)


had_variables, bnv_variables, angle = event_hypothesis([matched_hadb, matched_hadWq1, matched_hadWq2, matched_bnvq1, matched_bnvq2], matched_bnvlep)


In [ ]:
#values[2]

In [ ]:
for values in [had_variables]:#, bnv_variables]:
    plt.figure(figsize=(16,12))

    for i in range(len(values)):
        #x = ak.flatten(values[i]).to_numpy()   
        #print(type(values[i]))
        x = values[i]
        if type(x) == ak.highlevel.Array:
            x = values[i].to_numpy()
        #print(type(x))

        #print(len(x),x)
        #print(len(x[x==x]))
        x[x==-np.inf] = -999
        x[x==np.inf] = -999
        plt.subplot(5,5,i+1)
        if i>=15 and i<=17:
            plt.hist(x[x==x],bins=100,range=(0,250))
        elif i>=1 and i<=3:
            plt.hist(x[x==x],bins=100,range=(0,150))
        else:
            plt.hist(x[x==x],bins=100)

#plt.figure()
#plt.hist(np.cos(angle),bins=100);

In [ ]:
plt.hist(np.unique(had_variables[-3]),bins=100,range=(-3,3));#, range=(0,1200));

In [ ]:
plt.hist(had_variables[-6],bins=100,range=(0,250));
plt.hist(had_variables[-5],bins=100,range=(0,250),alpha=0.6);
plt.hist(had_variables[-4],bins=100,range=(0,250),alpha=0.4);

In [ ]:
plt.hist(had_variables[-9],bins=100,range=(0,150));
plt.hist(had_variables[-8],bins=100,range=(0,150),alpha=0.6);
plt.hist(had_variables[-7],bins=100,range=(0,150),alpha=0.4);

In [ ]:
x = values[0]

In [ ]:
x.to_numpy()

In [ ]:
x = ak.Array([12, 14, 19, 16]).to_numpy()
y = ak.Array([10, 15, 18, 13]).to_numpy()

In [ ]:
z = np.array([x,y])

In [ ]:
zt = z.transpose()

In [ ]:
idx = np.argsort(zt).transpose()

In [ ]:
idx

In [ ]:
x[idx[0]==0]